# ETL Steam Games

Importamos las librerias que utilizaremos en esta etapa del proyecto.

In [86]:
import pandas as pd
import numpy as np

## Extracción de Datos:

Hemos comenzado descomprimiendo los archivos json.gz con WinRAR, haciendo que los datos estén listos para su procesamiento en VS Code. Luego lo cargo en un DataFrame para su uso posterior.

In [87]:
df_game = pd.read_json('output_steam_games.json',lines=True)

In [88]:
df_game

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [89]:
df_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


## Transformación de Datos:

En segundo lugar, continuamos con la transformación. Esta consiste en limpiar, modificar y preparar los datos para su uso posterior. Eso incluye eliminar datos nulos, convertir tipos de datos, normalizar columnas, desanidar listas, etc.

### DATOS NULOS y COLUMNAS IRRELEVANTES

Ahora que se la magnitud del dataset empiezo a depurarlo. En primer lugar voy a cheuqear si hay o no **id** que tengan **valor nulo**, dado que eso me podria traer problemas al relacionar mis datos con los otros dataset. De existir **id** sin valor, entonces eliminaria esas filas.

In [90]:
id_null = df_game['id'].isnull().sum()
print(f'El dataset tiene {id_null} filas sin el dato id')

if id_null > 0:
    df_game2 = df_game.dropna(subset=['id'])
    print(f'El nuevo dataset tiene {df_game2.shape[0]} filas.')


El dataset tiene 88312 filas sin el dato id
El nuevo dataset tiene 32133 filas.


A grandes rasgos notamos  en el DataFrame original (dfgame) que al pedir la información, hay una gran cantidad de valores nulos. Eso se desprende de ver la cantidad de filas (120.445), y que en cada una de las columnas los valores no nulos llegan a ser como mucho un cuarto del total. Y al eliminar las filas con id nulo probablemente eliminamos tambien las filas en donde todos los datos de la fila son nulos, dado que pueden existir filas con **id** con **valor nulo** pero toda fila que tenga **todos** sus campos con **valor nulo** tendrá indefectiblemente **id** con **valor nulo** y por ende ya quedan eliminados en el nuevo dataframe. Por ello para seguir reduciendo el tamaño del dataframe, ahora eliminaremos las columnas irrelevantes para nuestro proyecto.

In [91]:
columns_relevant = ['id', 'genres', 'release_date', 'price', 'developer']
df_game3 = df_game2[columns_relevant]


In [92]:
df_game3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32133 non-null  float64
 1   genres        28851 non-null  object 
 2   release_date  30067 non-null  object 
 3   price         30756 non-null  object 
 4   developer     28835 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


En esta oportunidad revisare si hay filas cuyo **id** este presente pero que el **resto de los campos** en las columnas que me interesa conservar para luego desarrollar mis endpoints esten **todos nulos**.

In [93]:
columns_without_id = [col for col in df_game3.columns if col != 'id']
rows_null_with_id = df_game3[columns_without_id].isnull().all(axis=1).sum()
print(f'El dataset tiene {rows_null_with_id} filas con solo el dato id y el resto de campos con valor nulo')
if rows_null_with_id > 0:
    df_game4 = df_game3.dropna(subset=columns_without_id, how='all')
    print(f'El nuevo dataset tiene {df_game4.shape[0]} filas.')

El dataset tiene 116 filas con solo el dato id y el resto de campos con valor nulo
El nuevo dataset tiene 32017 filas.


In [94]:
df_game4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32017 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32017 non-null  float64
 1   genres        28851 non-null  object 
 2   release_date  30067 non-null  object 
 3   price         30756 non-null  object 
 4   developer     28835 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


### DATOS DUPLICADOS

A continuación revisamos si hay **datos duplicados**. Empezaremos por los id, dado que deben ser únicos. Y a contnuación chequeamos si hay filas que aún con id diferente comparten misma data en todo elresto de sus campos (eso podria deberse a error humano de dobles cargas por ejemplo)

In [95]:
df_game5 = df_game4.drop_duplicates(subset=['id'])
df_game5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32016 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32016 non-null  float64
 1   genres        28850 non-null  object 
 2   release_date  30066 non-null  object 
 3   price         30755 non-null  object 
 4   developer     28834 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [96]:
key_columns = ['id', 'release_date', 'developer']
df_game6 = df_game5.drop_duplicates(subset=key_columns)
print(f'El nuevo dataset tiene {df_game6.shape[0]} filas. Es decir lo mismo que el datset anterior, por ende no se justifica usar este nuevo.')


El nuevo dataset tiene 32016 filas. Es decir lo mismo que el datset anterior, por ende no se justifica usar este nuevo.


### CALIDAD DE DATOS

In [97]:
df_game5.dtypes

id              float64
genres           object
release_date     object
price            object
developer        object
dtype: object

A primera vista, podemos observar que hay datos que conviene convertir para tabajar con ellos, como ser la fecha, el id o ep precio. Iremos paso a paso.

**title**, **genres** y **developer** son object y no deben ser modificados para nuestros proyecto.           

**id**: lo covertiremos de float a int dado que al ser valores únicos se trata de un valor discreto (indivisible y no fraccionado).

In [98]:

# Asegurar que 'id' esté en formato int32 correctamente
df_game5 = df_game5.copy()  # Crear una copia para evitar el SettingWithCopyWarning
df_game5['id'] = df_game5['id'].astype('Int32')
df_game5.dtypes

id               Int32
genres          object
release_date    object
price           object
developer       object
dtype: object

In [99]:
df_game5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32016 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32016 non-null  Int32 
 1   genres        28850 non-null  object
 2   release_date  30066 non-null  object
 3   price         30755 non-null  object
 4   developer     28834 non-null  object
dtypes: Int32(1), object(4)
memory usage: 1.4+ MB


**release_date**: lo convertimos de object a date, pero primero debemos realizar ciertos pasos extras

In [100]:
import re

# Definir el patrón de búsqueda y la función para extraer el año
digits = r'(\d{4})' 

def year(string: str) -> str | None:
    num = re.search(digits, str(string))
    if num:
        return num.group(1)
    else:
        return None
# Remplazamos release date por el valor del año correspondiente en cadena y luego en la segunda linea lo convertimos en int32 (un numero entero discreto)
df_game5['release_date'] = df_game5['release_date'].apply(year)

df_game5['release_date'] = pd.to_numeric(df_game5['release_date'], errors='coerce').astype('Int32')
print(df_game5.info())


<class 'pandas.core.frame.DataFrame'>
Index: 32016 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32016 non-null  Int32 
 1   genres        28850 non-null  object
 2   release_date  29965 non-null  Int32 
 3   price         30755 non-null  object
 4   developer     28834 non-null  object
dtypes: Int32(2), object(3)
memory usage: 1.3+ MB
None


In [101]:
years_table = df_game5['release_date'].unique()
print("Los años que figuran en la lista son:", years_table)

year_null = df_game5['release_date'].isna().sum()
print("Cantidad de valores nulos en la columna 'release_date':", year_null)

Los años que figuran en la lista son: <IntegerArray>
[2018, 2017, <NA>, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002, 2000, 1995,
 1996, 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992, 1989, 2010, 2011, 2013,
 2012, 2014, 1983, 1984, 2015, 1990, 1988, 1991, 1985, 1982, 1987, 1981, 1986,
 2021, 5275, 2019, 1975, 1970, 1980]
Length: 45, dtype: Int32
Cantidad de valores nulos en la columna 'release_date': 2051


In [102]:
# Identificamos cuantos y en que filas estan los valores de año 5275, dado que es un año que no ha ocurrido aún.
year_5275 = df_game5[df_game5['release_date'] == 5275]

print(year_5275)

            id                         genres  release_date price developer
101738  710190  [Casual, Indie, Early Access]          5275  None    一次元创作组


In [103]:
# Procedemos a eliminar el dato para que no modifique radicalmente nuestras estadisticas, siendo que probablemente se trate de un error, y al ser un solo se justifica su eliminación.
df_game7 = df_game5.drop(101738)
print(df_game7.info())
years_table2 = df_game7['release_date'].unique()
print("Los años que figuran en la lista son:", years_table2)

<class 'pandas.core.frame.DataFrame'>
Index: 32015 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32015 non-null  Int32 
 1   genres        28849 non-null  object
 2   release_date  29964 non-null  Int32 
 3   price         30755 non-null  object
 4   developer     28833 non-null  object
dtypes: Int32(2), object(3)
memory usage: 1.3+ MB
None
Los años que figuran en la lista son: <IntegerArray>
[2018, 2017, <NA>, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002, 2000, 1995,
 1996, 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992, 1989, 2010, 2011, 2013,
 2012, 2014, 1983, 1984, 2015, 1990, 1988, 1991, 1985, 1982, 1987, 1981, 1986,
 2021, 2019, 1975, 1970, 1980]
Length: 44, dtype: Int32


**price**: lo convertimos de object a float dado que se trata de un tipo de dato continuo y por ende es el formato más adecuado.

Dado que uno de los endpoints a desarrollar (developer) menciona que existe un porcentaje de contenido Free  y al chequear el dicionario de Datos STEAM efectivamente figura más de una categoría con aquella palabra, entendemos que si queremos convertir a la categoría price en float, deberemos pasar los datos que expresen Free a un valor númerico, que en este caso sería su equivalente: 0.

In [104]:
# Contar valores nulos en la columna 'price'
price_null = df_game7['price'].isna().sum()
print(f"Total de valores nulos en 'price': {price_null}")

# Conteo de valores no numéricos en la columna price
no_num_value = df_game7['price'][~df_game7['price'].apply(pd.to_numeric, errors='coerce').notnull()]
total_types_no_num = no_num_value.value_counts()
print("Valores no numéricos detallados en 'price':")
print(total_types_no_num)

Total de valores nulos en 'price': 1260
Valores no numéricos detallados en 'price':
price
Free                             905
Free to Play                     520
Free To Play                     462
Free Mod                           4
Free Demo                          3
Play for Free!                     2
Third-party                        2
Play Now                           2
Play WARMACHINE: Tactics Demo      1
Install Now                        1
Install Theme                      1
Free HITMAN™ Holiday Pack          1
Play the Demo                      1
Starting at $499.00                1
Starting at $449.00                1
Free to Try                        1
Free Movie                         1
Free to Use                        1
Name: count, dtype: int64


In [105]:
# Convertir valores no numéricos 'Free' a '0'
df_game7['price'] = df_game7['price'].replace(to_replace=r'Free.*', value='0', regex=True)
df_game7['price'] = df_game7['price'].replace('Free HITMAN™ Holiday Pack', '0')

# Reemplazar valores no numéricos específicos
df_game7['price'] = df_game7['price'].replace('Play Now', '0')
df_game7['price'] = df_game7['price'].replace('Starting at $499.00', '499.00')
df_game7['price'] = df_game7['price'].replace('Starting at $449.00', '449.00')

# Convertir valores no numéricos restantes a NaN
df_game7['price'] = df_game7['price'].replace(['Play WARMACHINE: Tactics Demo', 'Install Now', 'Install Theme', 'Play the Demo', 'Third-party'], pd.NA)

# Convertir 'price' a numérico
df_game7['price'] = pd.to_numeric(df_game7['price'], errors='coerce')

# Calcular y reemplazar nulos con la mediana de 'price' para que así no afecten luego en el endpoint developers
median_price = df_game7['price'].median()
df_game7['price'] = df_game7['price'].fillna(median_price)

# Verificar la conversión
print(f"Mediana de 'price': {median_price}")
print(df_game7['price'].dtypes)


Mediana de 'price': 4.99
float64


In [106]:
df_game7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32015 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32015 non-null  Int32  
 1   genres        28849 non-null  object 
 2   release_date  29964 non-null  Int32  
 3   price         32015 non-null  float64
 4   developer     28833 non-null  object 
dtypes: Int32(2), float64(1), object(2)
memory usage: 1.3+ MB


In [107]:
values_of_price = df_game7['price'].unique()
values_of_price

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00, 3.9900e+00,
       9.9900e+00, 1.8990e+01, 2.9990e+01, 1.0990e+01, 1.5900e+00,
       1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00, 6.9900e+00,
       7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00, 1.2990e+01,
       5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00, 2.4990e+01,
       1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01, 1.3990e+01,
       3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01, 9.9990e+01,
       1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01, 4.9990e+01,
       5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01, 1.1999e+02,
       1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01, 8.9990e+01,
       9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01, 5.4990e+01,
       7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02, 1.2900e+00,
       3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01, 4.9000e+01,
       2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01, 4.4900e+00,
       2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e+02, 1.9500e

**developer**: si bien no hay que modificar este dato, al analizarlo, notamos que hay varios valores NonType que modificaremos con La Palabra "Unkown" para que no afecte luego mi trabajo en otras instancias.

In [108]:
print(df_game7['developer'].apply(type).value_counts())



developer
<class 'str'>         28833
<class 'NoneType'>     3182
Name: count, dtype: int64


In [109]:
# Reemplazar valores None en 'developer' con 'Unkown'
df_game7['developer'] = df_game7['developer'].fillna('Unkown')

# Verificar que el reemplazo se ha realizado correctamente
print(df_game7['developer'].apply(type).value_counts())


developer
<class 'str'>    32015
Name: count, dtype: int64


**genres**: Por último, encontramos que se trata de un dato anidado, así que deberemos trabajar sobre él.

In [110]:
# Desanidar la columna 'genres'
df_game8 = df_game7.explode('genres')

# Verificar el resultado de la desanidación
print(df_game8.head(10))

           id        genres  release_date  price         developer
88310  761140        Action          2018   4.99         Kotoshiro
88310  761140        Casual          2018   4.99         Kotoshiro
88310  761140         Indie          2018   4.99         Kotoshiro
88310  761140    Simulation          2018   4.99         Kotoshiro
88310  761140      Strategy          2018   4.99         Kotoshiro
88311  643980  Free to Play          2018   0.00  Secret Level SRL
88311  643980         Indie          2018   0.00  Secret Level SRL
88311  643980           RPG          2018   0.00  Secret Level SRL
88311  643980      Strategy          2018   0.00  Secret Level SRL
88312  670290        Casual          2017   0.00      Poolians.com


In [111]:
# Verificar que la columna 'genres' ahora esté desanidada correctamente
unique_genres = df_game8['genres'].unique()
print(f"Géneros únicos: {unique_genres}")

Géneros únicos: ['Action' 'Casual' 'Indie' 'Simulation' 'Strategy' 'Free to Play' 'RPG'
 'Sports' 'Adventure' None 'Racing' 'Early Access' 'Massively Multiplayer'
 'Animation &amp; Modeling' 'Video Production' 'Utilities'
 'Web Publishing' 'Education' 'Software Training'
 'Design &amp; Illustration' 'Audio Production' 'Photo Editing'
 'Accounting']


In [112]:
df_game8['genres'] = df_game8['genres'].replace('Animation &amp; Modeling', 'Animation and Modeling')
df_game8['genres'] = df_game8['genres'].replace('Design &amp; Illustration', 'Design and Illustration')

# Verificamos los cambios hechos
unique_genres = df_game8['genres'].unique()
print(f"Géneros únicos: {unique_genres}")

Géneros únicos: ['Action' 'Casual' 'Indie' 'Simulation' 'Strategy' 'Free to Play' 'RPG'
 'Sports' 'Adventure' None 'Racing' 'Early Access' 'Massively Multiplayer'
 'Animation and Modeling' 'Video Production' 'Utilities' 'Web Publishing'
 'Education' 'Software Training' 'Design and Illustration'
 'Audio Production' 'Photo Editing' 'Accounting']


### Cuidado Estético y Reorganización de columnas

In [113]:
# Normalizar nombres de columnas a español
new_column_names = {
    'genres': 'genero',
    'id': 'id_steam',
    'release_date': 'fecha',
    'price': 'precio',
    'developer': 'desarrollador'
}
df_game8 = df_game8.rename(columns=new_column_names)
df_game8.info()


<class 'pandas.core.frame.DataFrame'>
Index: 74714 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_steam       74714 non-null  Int32  
 1   genero         71548 non-null  object 
 2   fecha          72444 non-null  Int32  
 3   precio         74714 non-null  float64
 4   desarrollador  74714 non-null  object 
dtypes: Int32(2), float64(1), object(2)
memory usage: 3.0+ MB


In [114]:
reorganize = ['id_steam', 'desarrollador', 'genero','fecha', 'precio'] 

# Reordenar las columnas para una lectura más sencilla
df_game9 = df_game8[reorganize]

df_game9.head()

,id_steam,desarrollador,genero,fecha,precio
88310,761140,Kotoshiro,Action,2018,4.99
88310,761140,Kotoshiro,Casual,2018,4.99
88310,761140,Kotoshiro,Indie,2018,4.99
88310,761140,Kotoshiro,Simulation,2018,4.99
88310,761140,Kotoshiro,Strategy,2018,4.99


In [115]:
# Eliminar filas duplicadas en caso de que existan
df_game9 = df_game9.drop_duplicates()

# Verificar el DataFrame después de eliminar duplicados, como se ve en el resultado no hay duplicados en esta altura.
print(df_game9.info())


<class 'pandas.core.frame.DataFrame'>
Index: 74714 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_steam       74714 non-null  Int32  
 1   desarrollador  74714 non-null  object 
 2   genero         71548 non-null  object 
 3   fecha          72444 non-null  Int32  
 4   precio         74714 non-null  float64
dtypes: Int32(2), float64(1), object(2)
memory usage: 3.0+ MB
None


## Carga de Datos (Load)

En esta fase final del proceso ETL, los datos limpios y transformados se cargan en un sistema de almacenamiento o base de datos de destino. En nuestro caso utilizaremos Parquet dada su eficiencia. Yquedan ya listos para ser usados por FastAPI y Render.

In [116]:
import pyarrow
import fastparquet

In [117]:
# Guardar el DataFrame en formato CSV
df_game9.to_csv('output_steam_games.csv', index=False)
print("Archivo CSV guardado exitosamente.")

# Guardar el DataFrame en formato Parquet
df_game9.to_parquet('output_steam_games.parquet', index=False, engine='pyarrow', compression='snappy')
print("Archivo Parquet guardado exitosamente.")

Archivo CSV guardado exitosamente.
Archivo Parquet guardado exitosamente.


### Sub Datasets

Para poder trabajar con mayor agilidad crearemos ciertos subdatasets que apuntaran en definitiva a agilizar a la hora de utilizar los enpoints.

**Endpoint 1: developer.**

Filtra y retorna **cantidad de items** y **porcentaje** de contenido **"Free"** por **año** según empresa **desarrolladora**. Por ende eliminamos genero dado que no se utiliza en este punto.

In [118]:
df_endpoint1 = df_game9.drop(columns=['genero']).drop_duplicates(subset='id_steam')
print(df_endpoint1)

#Exportamos el dataset para usar en el ednpoint developer a Parquet
df_endpoint1.to_parquet('endpoint1.parquet', index=False, engine='pyarrow', compression='snappy')
print("Dataset específico para Endpoint 1 guardado exitosamente.")


        id_steam             desarrollador  fecha  precio
88310     761140                 Kotoshiro   2018    4.99
88311     643980          Secret Level SRL   2018    0.00
88312     670290              Poolians.com   2017    0.00
88313     767400                      彼岸领域   2017    0.99
88314     773570                    Unkown   <NA>    2.99
...          ...                       ...    ...     ...
120440    773640        Nikita "Ghost_RUS"   2018    1.99
120441    733530                    Sacada   2018    4.99
120442    610660  Laush Dmitriy Sergeevich   2018    1.99
120443    658870             xropi,stev3ns   2017    4.99
120444    681550                    Unkown   <NA>    4.99

[32015 rows x 4 columns]
Dataset específico para Endpoint 1 guardado exitosamente.


**Muestra Aleatoria**

En ciertas ocasiones puede ser de gran utilidad una muestra más pequeña de nuestro dataset para agilizar el trabajo. Y para que sea representativa la haremos aleatoria

In [119]:
# Selección aleatoria de 30,000 filas
df_random_sample = df_game9.sample(n=30000, random_state=42)
print(df_random_sample.head())
print(df_random_sample.info())

        id_steam               desarrollador     genero  fecha  precio
102662    633060             Osmotic Studios  Adventure   <NA>    4.99
104454    613050              Selenion Games      Indie   2017    4.99
116570    301830  Behaviour Interactive Inc.  Adventure   2012   19.99
89782     241320                    Ludosity      Indie   2013    9.99
117760    275050          Zaxis Games,B-evil      Indie   2014    0.99
<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 102662 to 108179
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_steam       30000 non-null  Int32  
 1   desarrollador  30000 non-null  object 
 2   genero         28721 non-null  object 
 3   fecha          29102 non-null  Int32  
 4   precio         30000 non-null  float64
dtypes: Int32(2), float64(1), object(2)
memory usage: 1.2+ MB
None


In [120]:
df_random_sample.to_parquet('random_sample.parquet', index=False, engine='pyarrow', compression='snappy')
print("Muestra aleatoria para consultas guardada exitosamente.")


Muestra aleatoria para consultas guardada exitosamente.


**Endpoint 6: recomendacion_juego**

Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado. Par ello debemos usar un par de columnas que habiamos dejado de lado por ende rettomamos el dataframe ante de seleccionar columnas, para hacer una nueva selección que en esta oportunidad se ajuste a nuestros requerimientos. id, genres, tags, price

In [121]:
columns_relevant2 = ['id', 'title', 'genres', 'tags', 'specs']
df_endpoint6 = df_game2[columns_relevant2]

print(df_endpoint6.info())

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32133 non-null  float64
 1   title   30084 non-null  object 
 2   genres  28851 non-null  object 
 3   tags    31971 non-null  object 
 4   specs   31464 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB
None


In [122]:
# Asegurar que 'id' esté en formato int32 correctamente
df_endpoint6 = df_endpoint6.copy()  # Crear una copia para evitar el SettingWithCopyWarning
df_endpoint6['id'] = df_endpoint6['id'].astype('Int32')
df_endpoint6.dtypes

id         Int32
title     object
genres    object
tags      object
specs     object
dtype: object

In [123]:
# Normalizar nombres de columnas a español
new_column_names2 = {
    'genres': 'genero',
    'id': 'id_steam',
    'tags': 'etiqueta',
    'title': 'titulo',
    'specs': 'especificaciones'
}
df_endpoint6 = df_endpoint6.rename(columns=new_column_names2)
df_endpoint6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_steam          32133 non-null  Int32 
 1   titulo            30084 non-null  object
 2   genero            28851 non-null  object
 3   etiqueta          31971 non-null  object
 4   especificaciones  31464 non-null  object
dtypes: Int32(1), object(4)
memory usage: 1.4+ MB


In [124]:
df_endpoint6.head()

,id_steam,titulo,genero,etiqueta,especificaciones
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player]
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla..."
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla..."
88313,767400,弹炸人2222,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player]
88314,773570,None,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V..."


In [125]:
# Copiar el DataFrame
df_endpoint6_final = df_endpoint6.copy()

# Seleccionar el primer género, etiqueta y especificación principal de cada ítem

df_endpoint6_final['genero'] = df_endpoint6['genero'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_endpoint6_final['etiqueta'] = df_endpoint6['etiqueta'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_endpoint6_final['especificaciones'] = df_endpoint6['especificaciones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

# Conteo de valores nulos
null_counts = df_endpoint6_final.isnull().sum()
print("Conteo de valores nulos antes de eliminar:")
print(null_counts)

# Eliminar filas con valores nulos
df_endpoint6_final2 = df_endpoint6_final.dropna()

# Verificar el DataFrame resultante
print(df_endpoint6_final2.head(10))

Conteo de valores nulos antes de eliminar:
id_steam               0
titulo              2049
genero              3282
etiqueta             162
especificaciones     669
dtype: int64
       id_steam                         titulo        genero      etiqueta  \
88310    761140            Lost Summoner Kitty        Action      Strategy   
88311    643980                      Ironbound  Free to Play  Free to Play   
88312    670290        Real Pool 3D - Poolians        Casual  Free to Play   
88313    767400                        弹炸人2222        Action        Action   
88315    772540          Battle Royale Trainer        Action        Action   
88316    774276   SNOW - All Access Basic Pass  Free to Play  Free to Play   
88317    774277     SNOW - All Access Pro Pass  Free to Play  Free to Play   
88318    774278  SNOW - All Access Legend Pass  Free to Play  Free to Play   
88319    768800                           Race        Casual         Indie   
88320    768570                Uncanny 

In [126]:
# Exportar el dataset a Parquet
df_endpoint6_final2.to_parquet('recomendacionjuego.parquet', index=False, engine='pyarrow', compression='snappy')
print("Dataset específico para recomendaciones guardado exitosamente.")

Dataset específico para recomendaciones guardado exitosamente.
